#### Feature engineering and selection of features begins

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [85]:
import os
import dotenv
import psycopg2

# -----------------------------------
# Load environment variables
# -----------------------------------
dotenv.load_dotenv()

# -----------------------------------
# Create connection
# -----------------------------------
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USERNAME"),
    password=os.getenv("DB_PASSWORD"),
    port=os.getenv("DB_PORT")
)

print("Connection established.")


Connection established.


In [86]:
import pandas as pd

query = "SELECT * FROM clean.train;"

chunks = []

for chunk in pd.read_sql(query, conn, chunksize=50000):
    chunks.append(chunk)

dataset_train = pd.concat(chunks, ignore_index=True)

dataset_train.head()


C:\Users\aadit\AppData\Local\Temp\ipykernel_30268\1467927621.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.read_sql(query, conn, chunksize=50000):


,store,dept,date,weekly_sales,isholiday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [87]:
dataset_train.isna().sum()

store           0
dept            0
date            0
weekly_sales    0
isholiday       0
dtype: int64

In [88]:
query = "SELECT * FROM clean.features_stores_merged;"

chunks = []

for chunk in pd.read_sql(query, conn, chunksize=50000):
    chunks.append(chunk)

features_stores_merged = pd.concat(chunks, ignore_index=True)

features_stores_merged.head()

C:\Users\aadit\AppData\Local\Temp\ipykernel_30268\3109442085.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.read_sql(query, conn, chunksize=50000):


,store,date,temperature,fuel_price,markdown1,markdown2,markdown3,markdown4,markdown5,total_markdown,cpi,unemployment,isholiday,max_weekly_sales,markdown_available,type,size,quarter
0,1,2010-02-05,42.31,2.572,0.0,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,139884.94,0,A,151315,2010Q1
1,1,2010-02-12,38.51,2.548,0.0,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True,143081.42,0,A,151315,2010Q1
2,1,2010-02-19,39.93,2.514,0.0,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False,135066.75,0,A,151315,2010Q1
3,1,2010-02-26,46.63,2.561,0.0,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False,125048.08,0,A,151315,2010Q1
4,1,2010-03-05,46.50,2.625,0.0,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,False,132945.44,0,A,151315,2010Q1


In [89]:
features_stores_merged.isna().sum()

store                    0
date                     0
temperature              0
fuel_price               0
markdown1                0
markdown2                0
markdown3                0
markdown4                0
markdown5                0
total_markdown           0
cpi                      0
unemployment             0
isholiday                0
max_weekly_sales      1755
markdown_available       0
type                     0
size                     0
quarter                  0
dtype: int64

In [90]:
subset=features_stores_merged[features_stores_merged['store']==1]
subset.shape

(182, 18)

In [91]:
subset[subset['max_weekly_sales'].isna()].shape

(39, 18)

In [92]:
39*45

1755

In [93]:
features_stores_merged['date'] = pd.to_datetime(features_stores_merged['date'])
features_stores_merged = features_stores_merged.sort_values(['store', 'date'])


In [94]:
import pandas as pd

results = []

for store, subset in features_stores_merged.groupby('store'):
    
    # Boolean mask where max_weekly_sales is null
    mask = subset['max_weekly_sales'].isna()
    
    # Identify groups of consecutive nulls
    # (~mask) flips nulls to False, so cumsum changes at each non-null
    null_groups = (mask != mask.shift()).cumsum()
    
    null_periods = subset[mask].groupby(null_groups).agg(
        start_date=('date', 'min'),
        end_date=('date', 'max')
    )
    
    null_periods['store'] = store
    results.append(null_periods.reset_index(drop=True))

# Combine all stores
null_ranges_df = pd.concat(results, ignore_index=True)
null_ranges_df = null_ranges_df[['store', 'start_date', 'end_date']]

null_ranges_df


,store,start_date,end_date
0,1,2012-11-02,2013-07-26
1,2,2012-11-02,2013-07-26
2,3,2012-11-02,2013-07-26
3,4,2012-11-02,2013-07-26
4,5,2012-11-02,2013-07-26
5,6,2012-11-02,2013-07-26
6,7,2012-11-02,2013-07-26
7,8,2012-11-02,2013-07-26
8,9,2012-11-02,2013-07-26
9,10,2012-11-02,2013-07-26
